In [ ]:
pip install datasets

In [ ]:
pip install evaluate

In [ ]:
# Transformers
from transformers import BartTokenizer, BartForConditionalGeneration            # BERT Tokenizer and architecture
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments               # fine-tune model
from transformers import pipeline                                               # pipeline
from transformers import DataCollatorForSeq2Seq                                 # DataCollator to batch the data
import torch                                                                    # PyTorch
import evaluate                                                                 # Hugging Face's library for model evaluation
from datasets import Dataset

import numpy as np
import pandas as pd
import nltk
# from textblob import TextBlob                                                   # fix spelling mistakes in texts
from sklearn.feature_extraction.text import TfidfVectorizer                     # identify the most common terms in the corpus
import re                                                                       # clean text data
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from datasets import load_dataset

# Load the multi_lexsum dataset
multi_lexsum = load_dataset("allenai/multi_lexsum", name="v20220616",trust_remote_code=True)

# Access the first instance of the validation set
example = multi_lexsum["validation"][0]

# Print the sources of the first example
print(example["sources"])

# Print the summaries of different lengths
for sum_len in ["long", "short", "tiny"]:
    print(f"{sum_len} summary: {example['summary/' + sum_len]}")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


['Page 1\nLEXSEE 2003 U.S. DIST. CT. PLEADINGS 3030 View U.S. District Court Opinion View Original Source Image of This Document\nSUSAN STOCKING, for herself and all other similarly situated, Plaintiff, v. AT&T CORP., Defendant.\nCase No. 03-0421-CV-W-HFS UNITED STATES DISTRICT COURT FOR THE WESTERN DISTRICT OF\nMISSOURI, WESTERN DIVISION 2003 U.S. Dist. Ct. Pleadings 3030; 2004 U.S. Dist. Ct. Pleadings LEXIS 9181\nJanuary 23, 2004 Complaint\nVIEW OTHER AVAILABLE CONTENT RELATED TO THIS DOCUMENT: U.S. District Court: Motion(s) COUNSEL: [**1] Sylvester "Sly" James, Jr. MO # 33617, Michael J. Mohlman KS # 19084, The Sly James Firm Trial Lawyers, P.C., 802 Broadway, 7th Floor, Kansas City, MO 64105, 816-472-6800, 816-472-6805 facsimile and Rex A. Sharp KS # 12350, Gunderson, Sharp & Rhein, P.C., 4121 W. 83rd St., Ste. 256, Prairie Village, KS 66208, 913-901-0500, 913-901-0419 facsimile and Rick D. Holtsclaw MO # 32866, Holtsclaw & Kendall, LC, 312 West 8th Street, Kansas City, MO 64105, 8

In [ ]:
import torch                                                                    # PyTorch

# Checking if GPU is available
if torch.cuda.is_available():
    print("GPU is available - Using GPU")
    device = torch.device('cuda')
else:
    print("GPU is not available - Using CPU")
    device = torch.device('cpu')

GPU is available - Using GPU


In [ ]:
# allow to fully read the dialogues and its summary
pd.set_option('display.max_colwidth', 1000)

In [ ]:
# Access the validation set
validation_set = multi_lexsum["validation"]

# Display the column names
print(validation_set.column_names)

['id', 'sources', 'summary/long', 'summary/short', 'summary/tiny']


In [ ]:
def display_feature_list(features, feature_type):

    '''
    This function displays the features within each list for each type of data
    '''

    print(f"\n{feature_type} Features: ")
    print(', '.join(features) if features else 'None')

def describe_df(df):

    global categorical_features
    categorical_features = [col for col in df.columns if df[col].dtype == 'object']

    print(f"\n{type(df).__name__} shape: {df.shape}")
    print(f"\n{df.shape[0]:,.0f} samples")
    print(f'\nMissing Data: \n{df.isnull().sum()}')
    print(f'\nDuplicates: {df.duplicated().sum()}')

    display_feature_list(categorical_features, 'Categorical')

    # print(f'\n{type(df).__name__} Head: \n')
    # display(df.head(5))
    # print(f'\n{type(df).__name__} Tail: \n')
    # display(df.tail(5))

In [ ]:
print(multi_lexsum)


DatasetDict({
    train: Dataset({
        features: ['id', 'sources', 'summary/long', 'summary/short', 'summary/tiny'],
        num_rows: 3177
    })
    validation: Dataset({
        features: ['id', 'sources', 'summary/long', 'summary/short', 'summary/tiny'],
        num_rows: 454
    })
    test: Dataset({
        features: ['id', 'sources', 'summary/long', 'summary/short', 'summary/tiny'],
        num_rows: 908
    })
})


In [ ]:
# Access the training, validation, and test sets
train_dataset = multi_lexsum["train"]
val_dataset = multi_lexsum["validation"]
test_dataset = multi_lexsum["test"]


In [ ]:
# Convert the Hugging Face Dataset to a Pandas DataFrame
train_df = train_dataset.to_pandas()
val_df = val_dataset.to_pandas()
test_df = test_dataset.to_pandas()


In [ ]:
# Step 1: Replace None values with np.nan
train_df.replace(to_replace=[None], value=np.nan, inplace=True)
val_df.replace(to_replace=[None], value=np.nan, inplace=True)
test_df.replace(to_replace=[None], value=np.nan, inplace=True)


In [ ]:
# Remove rows that have None or NaN values in 'summary/short' and 'summary/tiny' columns
train_df = train_df.dropna(subset=['summary/short', 'summary/tiny','summary/long'])
val_df = val_df.dropna(subset=['summary/short', 'summary/tiny','summary/long'])
test_df = test_df.dropna(subset=['summary/short', 'summary/tiny','summary/long'])

In [ ]:
train_df.head()

,id,sources,summary/long,summary/short,summary/tiny
7,PB-WV-0002,"[Case 3:13-cv-24068 Document 8 Filed 10/01/13 Page 1 of 30 PageID #: 16\n\nUNITED STATES DISTRICT COURT FOR THE SOUTHERN DISTRICT OF WEST VIRGINIA Huntington Division\n\nCASIE JO MCGEE and SARAH ELIZABETH ADKINS; JUSTIN MURDOCK and WILLIAM GLAVARIS; and NANCY ELIZABETH MICHAEL and JANE LOUISE FENTON, individually and as next friends of A.S.M., a minor child;\n\nPlaint[f{s,\nv.\nKAREN S. COLE, in her official capacity as CABELL COUNTY CLERK; and VERA J. MCCORMICK, in her official capacity as KANAWHA COlJNTY CLERK;\n\nCivil\n\nAction\n\nNo.\n\n3:13-cv-24068\n----\n\nDefendants.\n\nCOMPLAINT FOR DECLARATORY AND INJUNCTIVE RELIEF\n\nPlaintiffs CASTE JO MCGEE and SARAH ELIZABETH ADKINS; JUSTIN\n\nMURDOCK and WILLIAM GLAVARIS; and, NANCY ELIZABETH MICHAEL and JANE\n\nLOUISE FENTON, individually and as next friends of A.S.M., a minor child, all by and through\n\ntheir attorneys, file this Complaint against Defendants KAREN S. COLE, in her official capacity\n\nas CABELL COUNTY CLERK, and V...","On October 1, 2013, three same-sex couples and the minor child of one of the couples filed a lawsuit against Kanawha County, West Virginia in the U.S. District Court for the Southern District of West Virginia. The plaintiffs, represented by Lambda Legal Defense & Education Fund, asked the court to declare unconstitutional any West Virginia laws banning same-sex marriage, to enjoin West Virginia from refusing to recognize same-sex marriages undertaken in other states, and to award the plaintiffs reasonable attorney's fees and costs of suit.\n\nThe plaintiffs claimed that their rights under the Due Process and Equal Protection clauses of the United States Constitution were violated. They further asserted that the ban on same-sex marriage discriminated on the basis of sexual orientation, sex, and parental status. The plaintiffs also alleged that they were being denied a multitude of other social and legal rights that marriage provides.\n\nOn December 2, 2013, the Court (Judge Robert C...","On October 1, 2013, three same-sex couples and the minor child of one of the couples filed a lawsuit against Kanawha County, West Virginia. The plaintiffs asked the court to declare unconstitutional any West Virginia laws banning same-sex marriage, enjoin West Virginia from refusing to recognize same-sex marriages undertaken in other states, and to award the plaintiffs reasonable attorney's fees and costs of suit. The case was decided in favor of the plaintiffs, who were granted summary judgment in November 2014 and awarded attorneys’ fees, costs, and expenses.","Three same-sex couples, and the minor child of one of the couples won this same-sex marriage lawsuit in WV"
8,PN-MI-0008,"[Case 1:13-cv-00469-PLM Doc #1 Filed 05/01/13 Page 1 of 23 Page ID#1\n\nIN THE UNITED STATES DISTRICT COURT FOR THE WESTERN DISTRICT OF MICHIGAN\n\nGILBERT WEBER and TYRONE HIGHTOWER,\nPlaintiffs,\nvs.\nCITY OF GRAND RAPIDS; KEVIN BELK, Chief of Police of the Grand Rapids Police Department, in his official capacity; Officer JOHN GUERRERO, in his individual capacity; Officer THOMAS MCCARTHY, in his individual capacity; Officer GREGORY REKUCKI, in his individual capacity; and Officer ANTHONY LEONARD, in his individual capacity,\nDefendants.\n\nHon. Case No.\nJURY TRIAL DEMANDED\n\nCOMPLAINT\n\n Case 1:13-cv-00469-PLM Doc #1 Filed 05/01/13 Page 2 of 23 Page ID#2\nINTRODUCTORY STATEMENT 1. This civil rights case challenges the Grand Rapids Police Department’s ongoing practice of charging individuals with criminal trespassing at gas stations, bars and other commercial businesses open to the public, even though the individuals have done nothing wrong, and even though no one has asked the...","On May 1, 2013, two men who were arrested for trespassing on property open to the public filed this lawsuit in the U.S. District Court for the Western District of Michigan. The plaintiffs sued the City of Grand Rapids, its chief of poli

In [ ]:
# Print the number of samples in each set
print(f"Train set: {len(train_dataset)} samples")
print(f"Validation set: {len(val_dataset)} samples")
print(f"Test set: {len(test_dataset)} samples")

Train set: 3177 samples
Validation set: 454 samples
Test set: 908 samples


In [ ]:
train_df.info()  # Check data types of each column


<class 'pandas.core.frame.DataFrame'>
Index: 1129 entries, 7 to 3175
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             1129 non-null   object
 1   sources        1129 non-null   object
 2   summary/long   1129 non-null   object
 3   summary/short  1129 non-null   object
 4   summary/tiny   1129 non-null   object
dtypes: object(5)
memory usage: 52.9+ KB


In [ ]:
# Ensure that columns containing lists or arrays are converted to strings
train_df['sources'] = train_df['sources'].apply(str)
train_df['summary/long'] = train_df['summary/long'].apply(str)
train_df['summary/short'] = train_df['summary/short'].apply(str)
train_df['summary/tiny'] = train_df['summary/tiny'].apply(str)


In [ ]:
# Ensure that columns containing lists or arrays are converted to strings
val_df['sources'] = val_df['sources'].apply(str)
val_df['summary/long'] = val_df['summary/long'].apply(str)
val_df['summary/short'] = val_df['summary/short'].apply(str)
val_df['summary/tiny'] = val_df['summary/tiny'].apply(str)


In [ ]:
# Ensure that columns containing lists or arrays are converted to strings
test_df['sources'] = test_df['sources'].apply(str)
test_df['summary/long'] = test_df['summary/long'].apply(str)
test_df['summary/short'] = test_df['summary/short'].apply(str)
test_df['summary/tiny'] = test_df['summary/tiny'].apply(str)


In [ ]:

describe_df(train_df)
describe_df(val_df)
describe_df(test_df)




DataFrame shape: (1129, 5)

1,129 samples

Missing Data: 
id               0
sources          0
summary/long     0
summary/short    0
summary/tiny     0
dtype: int64

Duplicates: 0

Categorical Features: 
id, sources, summary/long, summary/short, summary/tiny

DataFrame shape: (161, 5)

161 samples

Missing Data: 
id               0
sources          0
summary/long     0
summary/short    0
summary/tiny     0
dtype: int64

Duplicates: 0

Categorical Features: 
id, sources, summary/long, summary/short, summary/tiny

DataFrame shape: (312, 5)

312 samples

Missing Data: 
id               0
sources          0
summary/long     0
summary/short    0
summary/tiny     0
dtype: int64

Duplicates: 0

Categorical Features: 
id, sources, summary/long, summary/short, summary/tiny


In [ ]:
# Drop 'id' and 'sources' columns from the train dataset
train_df = train_df.drop(['id','sources'], axis=1)

# Drop 'id' and 'sources' columns from the validation dataset
val_df = val_df.drop(['id','sources'], axis=1)

# Drop 'id' and 'sources' columns from the test dataset
test_df = test_df.drop(['id','sources'], axis=1)


In [ ]:

print("Long Summary:\n", train_df['summary/long'].iloc[1])
print("\nShort Summary:\n", train_df['summary/short'].iloc[1])
print("\nTiny Summary:\n", train_df['summary/tiny'].iloc[1])


Long Summary:
 On May 1, 2013, two men who were arrested for trespassing on property open to the public filed this lawsuit in the U.S. District Court for the Western District of Michigan. The plaintiffs sued the City of Grand Rapids, its chief of police, and two individual officers under 42 U.S.C. § 1983. The plaintiffs, represented by the National ACLU and ACLU of Michigan, asked the court for a declaratory judgment, damages, and injunctive relief concerning the use of "No Trespass Letters." The plaintiffs claimed that the City of Grand Rapids, its chief of police, and police officers violated their Fourth Amendment rights. 
  
According to the amended complaint, the Grand Rapids Police Department ("GRPD") had arrested individuals for trespassing based on a City trespass ordinance, under which the City solicits No Trespass Letters from area businesses indicating their intent to prosecute trespassers. The plaintiffs alleged they were arrested for trespassing while sitting in their vehi

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
print(model) # Visualizing model's architecture


BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [ ]:
summarizer = pipeline('summarization', model = 'facebook/bart-large-cnn')


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
text='''On May 1, 2013, two men who were arrested for trespassing on property open to the public filed this lawsuit in the U.S. District Court for the Western District of Michigan. The plaintiffs sued the City of Grand Rapids, its chief of police, and two individual officers under 42 U.S.C. § 1983. The plaintiffs, represented by the National ACLU and ACLU of Michigan, asked the court for a declaratory judgment, damages, and injunctive relief concerning the use of "No Trespass Letters." The plaintiffs claimed that the City of Grand Rapids, its chief of police, and police officers violated their Fourth Amendment rights.

According to the amended complaint, the Grand Rapids Police Department ("GRPD") had arrested individuals for trespassing based on a City trespass ordinance, under which the City solicits No Trespass Letters from area businesses indicating their intent to prosecute trespassers. The plaintiffs alleged they were arrested for trespassing while sitting in their vehicles in a business's parking lot, without any warning or complaint from the business itself. The plaintiffs claimed their arrests were without probable cause and therefore in violation of the Fourth Amendment. Additionally, the plaintiffs claimed that the ordinance violated the void-for-vagueness doctrine of the Due Process Clause of the Fourteenth
Amendment.

On December 3, 2013, the defendants filed a motion to dismiss the plaintiffs' claims for injunctive and declaratory relief. They argued that the plaintiffs lacked standing to seek declaratory or injunctive relief because they were not suffering an imminent threat of repeated future misconduct. Additionally, defendants argued that the plaintiffs' claims for declaratory and injunctive relief were not ripe because the City had changed the No Trespass Letters following the plaintiffs' filling of the Complaint. Defendants did not challenge the Court's jurisdiction to hear the plaintiffs' claim for damages.

On August 4, 2014, the Judge Paul L. Maloney dismissed the claims of one of the plaintiffs against the defendants according to the parties' stipulation. However, the other plaintiffs' claims remained.

While the motion to dismiss was still pending, the defendants and the plaintiffs both filed motions for summary judgment. In April 2015, the court postponed the trial date pending the resolution of multiple motions made by both the defendants and the plaintiffs.

Following a hearing on the defendants' motion to dismiss, as well as both parties' motions for summary judgment, Judge Maloney granted the defendants' motion to dismiss the plaintiffs' claims for injunctive and declaratory relief on June 21, 2017. 256 F.Supp.3d 742. The Court held that, because the plaintiffs had not alleged sufficient facts to present a threat of an imminent, as opposed to a speculative, injury, they lacked standing to seek declaratory and injunctive relief. The Court also noted that the City's changes to the No Trespass Letters following the filing of this case rendered the requests for prospective relief unripe. The Court did not, however, grant the defendants' motion to dismiss the plaintiffs' claim for damages.

Notably, the Michigan Court of Appeals addressed the same issue of whether the Grand Rapids ordinance is constitutional and held that it is unconstitutional. People v. Maggitt, 903 N.W.2d 868 (Mich. Ct. App. 2017). Following the Michigan Court of Appeals' ruling, the City effectively ended its practice of arresting individuals pursuant to the No Trespass Letters.

On October 17, 2018, Judge Maloney held that both sides were entitled to partial summary judgment. 407 F.Supp.3d 707.

The Court granted summary judgment for plaintiffs on their municipal liability claim against the City of Grand Rapids. The Court found that the City had an unconstitutional policy or custom whereby police officers arrested individuals for trespassing on property covered by a no-trespass letter without first informing the suspect that he or she must leave the property. However, the Court found that the plaintiffs had failed to show that the City's trespass ordinance was unconstitutionally vague.'''
summary1 = summarizer(text)
print(summary1)

[{'summary_text': 'On May 1, 2013, two men who were arrested for trespassing on property open to the public filed this lawsuit in the U.S. District Court for the Western District of Michigan. The plaintiffs sued the City of Grand Rapids, its chief of police, and two individual officers under 42 U.N.C. § 1983. On October 17, 2018, Judge Maloney held that both sides were entitled to partial summary judgment.'}]


In [ ]:
# Filter out rows where 'summary/long' or 'summary/short' are None
train_dataset = train_dataset.filter(lambda example: example['summary/long'] is not None and example['summary/short'] is not None)
val_dataset = val_dataset.filter(lambda example: example['summary/long'] is not None and example['summary/short'] is not None)
test_dataset = test_dataset.filter(lambda example: example['summary/long'] is not None and example['summary/short'] is not None)


In [ ]:
def preprocess_function(examples):
    # Ensure that the source text is 'summary/long' and the target is 'summary/short'
    # We skip rows with None or NaN values
    inputs = [doc for doc in examples["summary/long"] if doc is not None]
    targets = [summary for summary in examples["summary/short"] if summary is not None]

    # Tokenize the input text (summary/long)
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    # Setup the tokenizer for targets (summary/short)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/312 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
# Printing results
print('\n' * 3)
print('Preprocessed Training Dataset:\n')
print(train_dataset)
print('\n' * 2)
print('Preprocessed Test Dataset:\n')
print(val_dataset)
print('\n' * 2)
print('Preprocessed Validation Dataset:\n')
print(test_dataset)





Preprocessed Training Dataset:

Dataset({
    features: ['id', 'sources', 'summary/long', 'summary/short', 'summary/tiny', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2210
})



Preprocessed Test Dataset:

Dataset({
    features: ['id', 'sources', 'summary/long', 'summary/short', 'summary/tiny', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 312
})



Preprocessed Validation Dataset:

Dataset({
    features: ['id', 'sources', 'summary/long', 'summary/short', 'summary/tiny', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 616
})


In [ ]:
# Selecting a sample from the dataset
sample = train_dataset[0]

# Printing its features
print("input_ids:")                    # These are the token IDs
print(sample['input_ids'])
print("\n")
print("attention_mask:")               # indicates which tokens the model should pay attention to and which tokens should be ignored
print(sample['attention_mask'])
print("\n")
print("sample:")                       # token IDs obtained from the words and subwords in the summaries
print(sample['labels'])
print("\n")

input_ids:
[0, 4148, 772, 379, 6, 4013, 6, 5, 25235, 17820, 19469, 1463, 36, 9993, 4571, 43, 1658, 3235, 136, 446, 9, 3378, 6, 603, 482, 15, 4137, 9, 41, 3200, 54, 21, 2346, 2277, 142, 79, 21, 5283, 4, 9402, 5775, 8, 30272, 879, 32740, 3500, 13, 5, 3200, 36, 8529, 776, 1880, 6, 4660, 13, 3722, 4798, 6, 8, 21987, 8357, 238, 5, 27397, 4571, 1146, 3235, 223, 13497, 33559, 9, 5, 5280, 3941, 1783, 9, 17616, 13, 12286, 6886, 15, 5, 1453, 9, 2099, 4, 20, 27397, 4571, 67, 2952, 7, 5312, 63, 1042, 4, 50118, 50118, 42567, 940, 4778, 6, 5, 3200, 1658, 10, 4298, 7, 13192, 11, 5, 3235, 6, 61, 21, 6885, 4159, 71, 5, 675, 13, 3386, 16006, 1595, 396, 1160, 4, 20, 3200, 1146, 1449, 223, 13497, 33559, 8, 194, 488, 8, 2952, 12246, 5, 276, 3500, 25, 5, 27397, 4571, 6, 4682, 14, 5, 3674, 4010, 2952, 9493, 29963, 4, 50118, 50118, 42280, 5, 1799, 376, 7, 10, 4221, 1288, 6, 61, 5, 837, 36, 40145, 8051, 118, 229, 4, 5620, 387, 3876, 43, 2867, 25, 10, 7132, 20156, 15, 1133, 158, 6, 2338, 4, 20, 1110, 9, 5, 2015

In [ ]:
# Remove unnecessary columns from the tokenized dataset
columns_to_remove = ['id', 'sources', 'summary/long', 'summary/short', 'summary/tiny']
train_dataset.remove_columns(columns_to_remove)
val_dataset.remove_columns(columns_to_remove)
test_dataset.remove_columns(columns_to_remove)




Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 616
})

In [ ]:
# Instantiating Data Collator to batch the data
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
!pip install rouge_score


In [ ]:
# Loading ROUGE Score
metric = evaluate.load('rouge')

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred  # Obtaining predictions and true labels

    # Decoding predictions
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Handling masked tokens in labels (i.e., label = -100)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Compute the Rouge score
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # Check if `result` has the expected structure, and safely access fmeasure
    if isinstance(result, dict):
        result = {key: (value['fmeasure'] * 100 if isinstance(value, dict) and 'fmeasure' in value else value)
                  for key, value in result.items()}
    else:
        print("Unexpected result structure:", result)

    # Add mean-generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
# Defining parameters for training
training_args = Seq2SeqTrainingArguments(
    output_dir='Bart_law',
    evaluation_strategy="epoch",
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    seed=42,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    report_to="none"
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import EarlyStoppingCallback


In [ ]:
# Defining Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
# Training model
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,No log,1.083521,0.568000,0.380900,0.450000,0.525000,118.553600
2,1.028100,1.073198,0.578900,0.388500,0.457600,0.534700,126.527600
3,0.761300,1.097859,0.585200,0.394500,0.465700,0.540900,125.964300


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_

TrainOutput(global_step=1104, training_loss=0.8743328633515731, metrics={'train_runtime': 5613.4766, 'train_samples_per_second': 1.575, 'train_steps_per_second': 0.197, 'total_flos': 1.82136253292544e+16, 'train_loss': 0.8743328633515731, 'epoch': 3.992766726943942})

In [47]:
# Evaluating model performance on the tokenized validation dataset
validation = trainer.evaluate(eval_dataset = val_dataset)

print(validation)

{'eval_loss': 1.1003763675689697, 'eval_rouge1': 0.5791, 'eval_rouge2': 0.3897, 'eval_rougeL': 0.4642, 'eval_rougeLsum': 0.5391, 'eval_gen_len': 117.9968, 'eval_runtime': 452.8647, 'eval_samples_per_second': 0.689, 'eval_steps_per_second': 0.172, 'epoch': 3.992766726943942}


In [48]:
# Saving model
directory = "BART_Finetuned_law"
trainer.save_model(directory)

# Saving model tokenizer
tokenizer.save_pretrained(directory)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('BART_Finetuned_law/tokenizer_config.json',
 'BART_Finetuned_law/special_tokens_map.json',
 'BART_Finetuned_law/vocab.json',
 'BART_Finetuned_law/merges.txt',
 'BART_Finetuned_law/added_tokens.json',
 'BART_Finetuned_law/tokenizer.json')

In [49]:
from huggingface_hub import notebook_login

notebook_login()

In [51]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from huggingface_hub import HfApi, HfFolder, Repository

model_name = "hfwfjwjkj/bart_law_new_trained"

model = AutoModelForSeq2SeqLM.from_pretrained("BART_Finetuned_law")
tokenizer = AutoTokenizer.from_pretrained("BART_Finetuned_law")

model.push_to_hub(model_name)
tokenizer.push_to_hub(model_name)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hfwfjwjkj/bart_law_new_trained/commit/39b8af78c631826c5267a112e67668b17158b864', commit_message='Upload tokenizer', commit_description='', oid='39b8af78c631826c5267a112e67668b17158b864', pr_url=None, pr_revision=None, pr_num=None)

In [52]:
# loading the model
summarizer2 = pipeline("text2text-generation", model="hfwfjwjkj/bart_law_new_trained")

config.json:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/328 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [53]:
text='''On May 1, 2013, two men who were arrested for trespassing on property open to the public filed this lawsuit in the U.S. District Court for the Western District of Michigan. The plaintiffs sued the City of Grand Rapids, its chief of police, and two individual officers under 42 U.S.C. § 1983. The plaintiffs, represented by the National ACLU and ACLU of Michigan, asked the court for a declaratory judgment, damages, and injunctive relief concerning the use of "No Trespass Letters." The plaintiffs claimed that the City of Grand Rapids, its chief of police, and police officers violated their Fourth Amendment rights.

According to the amended complaint, the Grand Rapids Police Department ("GRPD") had arrested individuals for trespassing based on a City trespass ordinance, under which the City solicits No Trespass Letters from area businesses indicating their intent to prosecute trespassers. The plaintiffs alleged they were arrested for trespassing while sitting in their vehicles in a business's parking lot, without any warning or complaint from the business itself. The plaintiffs claimed their arrests were without probable cause and therefore in violation of the Fourth Amendment. Additionally, the plaintiffs claimed that the ordinance violated the void-for-vagueness doctrine of the Due Process Clause of the Fourteenth
Amendment.

On December 3, 2013, the defendants filed a motion to dismiss the plaintiffs' claims for injunctive and declaratory relief. They argued that the plaintiffs lacked standing to seek declaratory or injunctive relief because they were not suffering an imminent threat of repeated future misconduct. Additionally, defendants argued that the plaintiffs' claims for declaratory and injunctive relief were not ripe because the City had changed the No Trespass Letters following the plaintiffs' filling of the Complaint. Defendants did not challenge the Court's jurisdiction to hear the plaintiffs' claim for damages.

On August 4, 2014, the Judge Paul L. Maloney dismissed the claims of one of the plaintiffs against the defendants according to the parties' stipulation. However, the other plaintiffs' claims remained.

While the motion to dismiss was still pending, the defendants and the plaintiffs both filed motions for summary judgment. In April 2015, the court postponed the trial date pending the resolution of multiple motions made by both the defendants and the plaintiffs.

Following a hearing on the defendants' motion to dismiss, as well as both parties' motions for summary judgment, Judge Maloney granted the defendants' motion to dismiss the plaintiffs' claims for injunctive and declaratory relief on June 21, 2017. 256 F.Supp.3d 742. The Court held that, because the plaintiffs had not alleged sufficient facts to present a threat of an imminent, as opposed to a speculative, injury, they lacked standing to seek declaratory and injunctive relief. The Court also noted that the City's changes to the No Trespass Letters following the filing of this case rendered the requests for prospective relief unripe. The Court did not, however, grant the defendants' motion to dismiss the plaintiffs' claim for damages.

Notably, the Michigan Court of Appeals addressed the same issue of whether the Grand Rapids ordinance is constitutional and held that it is unconstitutional. People v. Maggitt, 903 N.W.2d 868 (Mich. Ct. App. 2017). Following the Michigan Court of Appeals' ruling, the City effectively ended its practice of arresting individuals pursuant to the No Trespass Letters.

On October 17, 2018, Judge Maloney held that both sides were entitled to partial summary judgment. 407 F.Supp.3d 707.

The Court granted summary judgment for plaintiffs on their municipal liability claim against the City of Grand Rapids. The Court found that the City had an unconstitutional policy or custom whereby police officers arrested individuals for trespassing on property covered by a no-trespass letter without first informing the suspect that he or she must leave the property. However, the Court found that the plaintiffs had failed to show that the City's trespass ordinance was unconstitutionally vague.'''
summary1 = summarizer2(text)
print(summary1)

[{'generated_text': "On May 1, 2013, two men who were arrested for trespassing on property open to the public filed this lawsuit in the U.S. District Court for the Western District of Michigan. The plaintiffs sued the City of Grand Rapids, its chief of police, and two individual officers, alleging violations of the Fourth Amendment and the Due Process Clause of the Fourteenth Amendment. The Court granted summary judgment for plaintiffs on their municipal liability claim against the Grand Rapids Police Department on October 17, 2018, finding that the City's trespass ordinance was unconstitutional."}]
